In [ ]:
import matplotlib.pyplot as plt
from mean_reversion_stragtegy_complex import *

In [ ]:
outdir = 'mr_results'
os.makedirs(outdir, exist_ok=True)

# --- single asset example ---
tick = 'SPY'
data = fetch_data([tick], period='5y', interval='1d')
close = data[tick]['close']

print('Running single-asset mean reversion on', tick)
res = single_asset_mean_reversion(close, ma_window=20, std_window=20, entry_z=2.0, exit_z=0.0,
                                    pct_per_trade=0.1, tc=0.0005, slippage=0.0002)
print('Metrics:', res['metrics'])
res['trades'].to_csv(os.path.join(outdir, f'{tick}_single_trades.csv'), index=False)
res['equity_curve'].to_csv(os.path.join(outdir, f'{tick}_single_equity.csv'))
plot_equity(res['equity_curve'], f'{tick} Single Asset MR Equity', fname=os.path.join(outdir, f'{tick}_single_equity.png'))

# save price+z
ma = close.rolling(20).mean()
sigma = close.rolling(20).std()
z = (close - ma) / sigma
plot_price_with_z(close, z, f'{tick} price + z-score', fname=os.path.join(outdir, f'{tick}_price_z.png'))

# --- pairs example ---
tick_x = 'XLF'
tick_y = 'KBE'
print('Fetching pair:', tick_x, tick_y)
pairdata = fetch_data([tick_x, tick_y], period='5y', interval='1d')
px = pairdata[tick_x]['close']
py = pairdata[tick_y]['close']

print('Running pairs mean reversion')
pres = pairs_mean_reversion(px, py, lookback=60, z_entry=2.0, z_exit=0.5,
                            pct_per_trade=0.1, tc=0.0005, slippage=0.0002)
print('Pair Metrics:', pres['metrics'])
pres['trades'].to_csv(os.path.join(outdir, f'{tick_x}_{tick_y}_pair_trades.csv'), index=False)
pres['equity_curve'].to_csv(os.path.join(outdir, f'{tick_x}_{tick_y}_pair_equity.csv'))
plot_equity(pres['equity_curve'], f'{tick_x}-{tick_y} Pairs MR Equity', fname=os.path.join(outdir, f'{tick_x}_{tick_y}_pair_equity.png'))

print('Done. Results saved to', outdir)
